In [1]:
import bt
import sys
import glob
import pickle
import numpy as np
import pandas as pd
import scipy.signal as signal
from scipy.stats import boxcox
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import coint, adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
%matplotlib inline

## Import Data

In [2]:
# concatenate & preprocessing data
dfList = []
for file in glob.glob(sys.path[0] + '/dataset/*.csv'):
    tmp = pd.read_csv(file, index_col=0)[['Value_Close']]
    tmp.rename(columns={'Value_Close': file.split('-')[0].split('/')[-1]}, inplace=True)
    tmp.index = pd.to_datetime(tmp.index)
    dfList.append(tmp)
df = pd.concat(dfList, axis=1)
df = df.resample('1D').last()
df.fillna(method='ffill', inplace=True)

### augmented Engle-Granger two-step cointegration test
[statsmodels.tsa.stattools.coint](https://www.statsmodels.org/stable/generated/statsmodels.tsa.stattools.coint.html)

In [3]:
stat, p, cp = coint(df[['FXF1']], df[['EXF1']])
print('p-value: %.5f\n' % p)
print('critical values \n%.5f at 1percent\n%.5f at 5percent\n%.5f at 10percent' 
      % (cp[0], cp[1], cp[2])
)

p-value: 0.02378

critical values 
-3.89795 at 1percent
-3.33697 at 5percent
-3.04504 at 10percent


### Augmented Dicky Fuller's Test

In [ ]:
_, p, _, _, _, _ = adfuller(df['FXF1'])
print('FXF1 adf p-value: %.5f\n' % p)
plot_acf(df['FXF1'])
plot_pacf(df['FXF1'])

FXF1 adf p-value: 0.05286



In [ ]:
_, p, _, _, _, _ = adfuller(df['EXF1'])
print('EXF1 adf p-value: %.5f\n' % p)
plot_acf(df['EXF1'])
plot_pacf(df['EXF1'])

### Cross Correlation Plot

In [ ]:
def ccf(x, y):
    if isinstance(x, (pd.Series, pd.DataFrame)):
        x = x.values
    if isinstance(y, (pd.Series, pd.DataFrame)):
        y = y.values
        
    nom = signal.correlate(x-np.mean(x), y-np.mean(y), method='direct')
    denom = np.std(x)*np.std(y)*len(x)
    return nom/denom

def plot_ccf(x, y, maxlags=20):
    result = ccf(x, y)
    lo = (len(result)-1)//2-maxlags #just get +/- 10 elements around lag 0
    hi = (len(result)-1)//2+(maxlags+1)

    #Make a plot like ccf
    f, ax = plt.subplots(figsize=(30, 10))
    ax.stem(np.arange(-1*maxlags,maxlags+1), result[lo:hi], linefmt='b-', markerfmt='bo', basefmt='r-')
    ax.set_xticks(np.arange(-1*maxlags,maxlags+1))
    plt.title('CCF - ' + x.name + ' vs. ' + y.name)
    plt.show()

In [ ]:
fx, _ = boxcox(df['FXF1'])
ex, _ = boxcox(df['EXF1'])
fx = pd.Series(fx, index=df.index, name='FXF1')
ex = pd.Series(ex, index=df.index, name='EXF1')
plot_ccf(fx, ex)

## Listing all Result of ExF strategy

In [ ]:
with open(sys.path[0] + '/result/ExF-2019-05-24 11:33:40.619394.pkl', 'rb') as f:
    ExF = pickle.load(f)

In [ ]:
# Backtest Statistics Table
'''
CAGR: compound annual growth rate
'''
ExF.display()

In [ ]:
# Equity Plot
ExF.plot()
plt.plot(plt.xlim(), plt.ylim(), 'k-', color = 'r')
plt.ylim(plt.ylim())
plt.xlim(plt.xlim())

In [ ]:
# return in each month
list(ExF.values())[0].return_table

In [ ]:
# return histogram
ExF.plot_histogram()

In [ ]:
ExF.plot_weights()
plt.title('Weights Plot (including total weights)')

ExF.plot_security_weights()
plt.title('Weights Plot (excluding total weights)')

In [ ]:
# DropDown Ratio Plot
ExF.prices.to_drawdown_series().plot(figsize=(20, 7))
plt.title('DropDown Ratio')

In [ ]:
# list all transactions
ExF.get_transactions().head(8)

## Listing All Result of EnF strategy

In [ ]:
with open(sys.path[0] + '/result/EnF-2019-05-24 11:32:54.964336.pkl', 'rb') as f:
    EnF = pickle.load(f)

In [ ]:
# Backtest Statistics Table
'''
CAGR: compound annual growth rate
'''
EnF.display()

In [ ]:
# Equity Plot
EnF.plot()
plt.plot(plt.xlim(), plt.ylim(), 'k-', color = 'r')
plt.ylim(plt.ylim())
plt.xlim(plt.xlim())

In [ ]:
# return in each month
list(EnF.values())[0].return_table

In [ ]:
# return histogram
EnF.plot_histogram()

In [ ]:
EnF.plot_weights()
plt.title('Weights Plot (including total weights)')

EnF.plot_security_weights()
plt.title('Weights Plot (excluding total weights)')

In [ ]:
# DropDown Ratio Plot
EnF.prices.to_drawdown_series().plot(figsize=(20, 7))
plt.title('DropDown Ratio')

In [ ]:
# list all transactions
EnF.get_transactions().head(8)

## Combine Stratgies

In [ ]:
with open(sys.path[0] + '/trade/EnF-2019-05-24 11:32:54.964336.pkl', 'rb') as f:
    Ntrade = pickle.load(f)
with open(sys.path[0] + '/trade/ExF-2019-05-24 11:33:40.619394.pkl', 'rb') as f:
    Xtrade = pickle.load(f)

In [ ]:
combine = bt.run(Ntrade, Xtrade)

In [ ]:
# Backtest Statistics Table
combine.display()

In [ ]:
combine.plot_correlation()

In [ ]:
# 組合策略
# 根據 1/波動率 給予權重
data = bt.merge(*[v.prices for v in combine.values()])
s = bt.Strategy('s', [bt.algos.SelectAll(),
                      bt.algos.WeighInvVol(),
                      bt.algos.Rebalance()])

# create and run
t = bt.Backtest(s, data)
res = bt.run(t)

In [ ]:
# Equity Plot
res.plot()
plt.plot(plt.xlim(), plt.ylim(), 'k-', color = 'r')
plt.ylim(plt.ylim())
plt.xlim(plt.xlim())

In [ ]:
# return in each month
list(res.values())[0].return_table

In [ ]:
# return histogram
res.plot_histogram()

In [ ]:
res.plot_weights()
plt.title('Weights Plot (including total weights)')

res.plot_security_weights()
plt.title('Weights Plot (excluding total weights)')

In [ ]:
# DropDown Ratio Plot
res.prices.to_drawdown_series().plot(figsize=(20, 7))
plt.title('DropDown Ratio')

In [ ]:
# list all transactions
res.get_transactions().head(8)